In [1]:
from ray.rllib.offline.json_reader import JsonReader
import numpy as np
import tensorflow as tf

In [2]:
input_reader = JsonReader("logs2")

2022-12-16 13:50:11,709	WARNING json_reader.py:238 -- Treating input directory as glob patterns: ['/tf/Cage/Notebooks/logs2/*.json', '/tf/Cage/Notebooks/logs2/*.zip']


In [4]:
sequence_length=10
num_episodes = 10
windows_per_episode = 100-sequence_length
state_len = 52
num_action = 145
encoding_len = state_len + num_action

data_points = num_episodes * windows_per_episode

X = np.zeros((data_points,sequence_length,encoding_len))
Y = np.zeros((data_points,state_len))

index = 0
for e in range(num_episodes):
    data = input_reader.next()
    data['obs'].shape
    for i in range(0, windows_per_episode):
        ns = data['obs'][i+1]
        if i < sequence_length:
            s = np.zeros((sequence_length, encoding_len))
            for k in range(0,i):
                s[k,:] = np.concatenate([data['obs'][k], np.zeros(num_action)])
        else:
            vec = np.zeros((sequence_length, num_action))
            vec[np.arange(sequence_length),data['actions'][i-sequence_length:i]] = 1
            s = np.concatenate([data['obs'][i-sequence_length:i], vec], axis=1)
            
        X[index,:,:] = s
        Y[index,:] = ns
        index += 1
index

900

In [16]:
import sys
import numpy
numpy.set_printoptions(threshold=1000)
print(X[10])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [6]:
print(Y[10])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 0. 1.]


In [7]:
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import Bidirectional


model = Sequential()
model.add(Bidirectional(LSTM(256, activation='relu', return_sequences=True), input_shape=(sequence_length, encoding_len)))
model.add(Bidirectional(LSTM(256, activation='relu', return_sequences=True)))
model.add(Flatten())
model.add(Dense(state_len, activation='sigmoid'))
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=False))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

print(model.summary())

model.fit(X, Y, epochs=100, validation_split=0.1, verbose=1, callbacks=[callback], batch_size=128)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 10, 512)          929792    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 10, 512)          1574912   
 nal)                                                            
                                                                 
 flatten (Flatten)           (None, 5120)              0         
                                

/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


810000/810000 [==============================] - 225s 278us/sample - loss: 0.0726 - val_loss: 0.0670
Epoch 2/100
810000/810000 [==============================] - 221s 273us/sample - loss: 0.0661 - val_loss: 0.0653
Epoch 3/100
810000/810000 [==============================] - 221s 273us/sample - loss: 0.0648 - val_loss: 0.0649
Epoch 4/100
810000/810000 [==============================] - 220s 271us/sample - loss: 0.0639 - val_loss: 0.0649
Epoch 5/100
810000/810000 [==============================] - 221s 273us/sample - loss: 0.0630 - val_loss: 0.0653
Epoch 6/100
810000/810000 [==============================] - 221s 272us/sample - loss: 0.0621 - val_loss: 0.0657


In [8]:
print(model.predict(np.array([X[10]])))

/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


[[9.60330013e-03 3.66523665e-07 3.50416371e-07 3.66764567e-07
  8.70442912e-02 2.31272148e-04 2.65252766e-05 3.17911647e-04
  1.87745094e-01 6.86138645e-02 2.40779453e-04 6.71900809e-02
  4.68524732e-02 3.84961255e-04 2.22684990e-04 2.70441128e-03
  1.85803883e-02 3.81132281e-07 3.73594816e-07 3.73049346e-07
  2.28173472e-02 3.58579200e-07 3.35807357e-07 3.69348982e-07
  2.01070644e-02 3.48535991e-07 3.77001413e-07 3.47464038e-07
  2.03166790e-02 4.10109569e-05 1.55242233e-04 6.27819623e-04
  7.47409835e-02 3.48344912e-07 3.56558729e-07 3.66423336e-07
  7.60118067e-02 2.35791376e-04 3.35789286e-03 9.79248762e-01
  8.97541344e-02 1.19584205e-04 2.01299918e-04 1.02442701e-03
  8.28449875e-02 1.57724455e-04 4.37784081e-07 1.43291114e-03
  1.05044916e-01 5.23213355e-04 4.93744174e-08 6.06363686e-03]]


In [9]:
print(np.round(model.predict(np.array([X[10]])))[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [10]:
print(Y[10])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [11]:
p = model.predict(np.array([X[10]]))[0]
p[np.where(Y[10]==1)]

array([0.07601181, 0.97924876], dtype=float32)

In [12]:
np.save('target.npy',X[10])
np.save('prediction.npy',p)